In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

### 0. 데이터 압축해제

In [ ]:
# Training Image
%cd /gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training/01.원천데이터
!unzip -qq "/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training(압축)/01.원천데이터/TS.zip"

In [ ]:
# Training Label
%cd /gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training/02.라벨링데이터
!unzip -qq "/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training(압축)/02.라벨링데이터/TL.zip"

In [ ]:
# Validation Image
%cd /gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Validation/01.원천데이터
!unzip -qq "/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Validation(압축)/01.원천데이터/VS.zip"

In [ ]:
# Validation Label
%cd /gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Validation/02.라벨링데이터
!unzip -qq "/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Validation(압축)/02.라벨링데이터/VL.zip"

### 0-1. 데이터 확인

In [ ]:
# 데이터 불러오기
import os
from PIL import Image
import json

# 데이터 폴더 경로
data_dir = '/gdrive/MyDrive/Final project/skindata/Validation'
raw_data_dir = os.path.join(data_dir, '01.원천데이터')
label_data_dir = os.path.join(data_dir, '02.라벨링데이터')

# 파일 리스트를 저장할 딕셔너리 초기화
images_dict = {}
json_dict = {}

# 원천 데이터 폴더 내의 모든 jpg 파일 탐색
for root, dirs, files in os.walk(raw_data_dir):
    for file in files:
        if file.endswith('.jpg'):
            file_path = os.path.join(root, file)
            file_name = os.path.splitext(file)[0]  # 확장자를 제외한 파일 이름
            images_dict[file_name] = Image.open(file_path)

# 라벨링 데이터 폴더 내의 모든 json 파일 탐색
for root, dirs, files in os.walk(label_data_dir):
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(root, file)
            file_name = os.path.splitext(file)[0]  # 확장자를 제외한 파일 이름
            with open(file_path, 'r', encoding='utf-8') as f:
                json_dict[file_name] = json.load(f)

In [ ]:
# 찾고자 하는 파일 이름
search_filename = '0001_01_Fb.jpg'

# filename으로 검색하여 항목 찾기
matching_entries = {key: value for key, value in json_dict.items() if value['info']['filename'] == search_filename}

# 결과를 하나의 딕셔너리로 묶기
combined_dict = {"total": matching_entries}

# 결과 출력
print(json.dumps(combined_dict, indent=4, ensure_ascii=False))

In [ ]:
len(images_dict), len(json_dict)

In [ ]:
images_dict['0001_01_F']

### 1. 데이터 폴더 정리
- ['1. 디지털카메라', '2. 스마트패드', '3. 스마트폰'] 폴더 안에 <br>-> 데이터 ['01.원천데이터','02.라벨링데이터']폴더로 이동
- ['1. 디지털카메라', '2. 스마트패드', '3. 스마트폰'] 폴더 삭제


In [ ]:
import os
import json
import pandas as pd
import numpy as np

In [ ]:
devices_name = ['1. 디지털카메라', '2. 스마트패드', '3. 스마트폰']
# 1. 라벨링 데이터 정리
def reorganize_labeling_data(base_path):
    for split in ['Validation', 'Training']:
        label_path = os.path.join(base_path, split, '02.라벨링데이터')
        if not os.path.exists(label_path):
            print(f"경로가 존재하지 않습니다: {label_path}")
            continue
        for device in devices_name:
            device_path = os.path.join(label_path, device)
            if not os.path.exists(device_path):
                print(f"경로가 존재하지 않습니다: {device_path}")
                continue
            for root, dirs, files in os.walk(device_path):
                for file in files:
                    if file.endswith('.json'):
                        src = os.path.join(root, file)
                        dst = os.path.join(label_path, file)
                        shutil.move(src, dst)

        # 빈 폴더 제거
        for device in devices_name:
            device_path = os.path.join(label_path, device)
            if os.path.exists(device_path):
                shutil.rmtree(device_path)

In [ ]:
# 2. 이미지 데이터 정리
def reorganize_image_data(base_path):
    for split in ['Validation', 'Training']:
        image_path = os.path.join(base_path, split, '01.원천데이터')
        if not os.path.exists(image_path):
            print(f"경로가 존재하지 않습니다: {image_path}")
            continue
        for device in devices_name:
            device_path = os.path.join(image_path, device)
            if not os.path.exists(device_path):
                print(f"경로가 존재하지 않습니다: {device_path}")
                continue
            for root, dirs, files in os.walk(device_path):
                for file in files:
                    if file.endswith('.jpg'):
                        src = os.path.join(root, file)
                        dst = os.path.join(image_path, file)
                        shutil.move(src, dst)

        # 빈 폴더 제거
        for device in devices_name:
            device_path = os.path.join(image_path, device)
            if os.path.exists(device_path):
                shutil.rmtree(device_path)

### 2. JSON형태의 라벨링 데이터를 DataFrame으로 변환

In [ ]:
# 3. 라벨링 데이터를 DataFrame으로 변환
def create_labeling_dataframe(base_path):
    data = []
    for split in ['Validation', 'Training']:
        label_path = os.path.join(base_path, split, '02.라벨링데이터')
        if not os.path.exists(label_path):
            print(f"경로가 존재하지 않습니다: {label_path}")
            continue
        for file in os.listdir(label_path):
            if file.endswith('.json'):
                with open(os.path.join(label_path, file), 'r') as f:
                    json_data = json.load(f)
                    row = {
                        'split': split,
                        'info': json.dumps(json_data.get('info', {})),
                        'images': json.dumps(json_data.get('images', {})),
                        'annotations': json.dumps(json_data.get('annotations', {})),
                        'equipment': json.dumps(json_data.get('equipment', {}))
                    }
                    data.append(row)

    df = pd.DataFrame(data)

    # JSON 문자열을 딕셔너리로 변환
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: x if isinstance(x, dict) else json.loads(x) if isinstance(x, str) else x)

    return df

In [ ]:
# 4. 이미지 자르기 및 저장
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def crop_and_save_images(base_path):

    for split in ['Validation', 'Training']:
        image_path = os.path.join(base_path, split, '01.원천데이터')
        label_path = os.path.join(base_path, split, '02.라벨링데이터')

        if not os.path.exists(image_path) or not os.path.exists(label_path):
            print(f"경로가 존재하지 않습니다: {image_path} 또는 {label_path}")
            continue

        for facepart in range(9):
            os.makedirs(os.path.join(base_path, f'{split}_cropped', str(facepart)), exist_ok=True)

        for file in os.listdir(image_path):
            if file.endswith('.jpg'):
                try:
                    img = Image.open(os.path.join(image_path, file))
                    img.verify()  # 이미지 파일 검증
                    img = Image.open(os.path.join(image_path, file))  # 다시 열기
                except Exception as e:
                    print(f"오류 발생: {file} - {str(e)}")
                    continue

                for json_file in os.listdir(label_path):
                    if json_file.startswith(file[:-4]) and json_file.endswith('.json'):
                        with open(os.path.join(label_path, json_file), 'r') as f:
                            json_data = json.load(f)

                        facepart = json_data['images']['facepart']
                        bbox = json_data['images']['bbox']

                        try:
                            if bbox is None:
                                # bbox가 None이면 전체 이미지 사용
                                bbox = [0, 0, img.width, img.height]
                            else:
                                # bbox 좌표가 문자열인 경우 정수로 변환
                                bbox = [int(float(coord)) for coord in bbox]

                            # bbox 좌표가 올바른지 확인
                            if bbox[2] <= bbox[0] or bbox[3] <= bbox[1]:
                                print(f"잘못된 bbox 좌표: {file} - {bbox}")
                                continue

                            cropped_img = img.crop(bbox)
                            cropped_img.save(os.path.join(base_path, f'{split}_cropped', str(facepart), f"{file[:-4]}_{facepart}.jpg"))
                        except Exception as e:
                            print(f"이미지 크롭 오류 발생: {file} - {str(e)}")
                            continue

In [ ]:
# 실행 코드
base_path = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터'  # 데이터 기본 경로

reorganize_labeling_data(base_path)
reorganize_image_data(base_path)
df = create_labeling_dataframe(base_path)
crop_and_save_images(base_path)

### 3. 데이터 프레임 전처리
- 데이터 프레임에 있는 None값 -> {}
- 딕셔너리 안에 있는 None값 -> 0(임의로 설정)

In [ ]:
# 5. 데이터 프레임 전처리

# 데이터 프레임 None 값을 {}로 변경
# 딕셔너리 안에 None 값을 0으로 바꾸는 함수 정의
def replace_none_with_zero(d):
    if isinstance(d, dict):
        return {k: (0 if v is None else v) for k, v in d.items()}
    return d

# 데이터 프레임의 모든 셀에 대해 함수 적용
df = df.applymap(replace_none_with_zero)

# 'equipment' 컬럼에 null 값을 빈 딕셔너리로 변경
df['equipment'] = df['equipment'].apply(lambda x: {} if pd.isna(x) else x)

print("데이터 정리 및 처리가 완료!")
df.head(3)

### 4. 데이터프레임을 csv로 저장

In [ ]:
# 데이터프레임을 csv로 저장
df.to_csv('data.csv', index=False)